<insert. Fannie Mae logo>

___________
# Mortgage Loan Default Classifier

### Problem Statement:
- Predict whether a mortgage loan will default using Fannie Mae Loan Performance data set from previous quarter

-----------

## EDA and Cleaning
___

In [1]:
# import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# define function to to allow user to assign train year and quarter of training data

In [3]:
# define function to load data set

def load_dataset(year=2005, quarter=1):

    # load acquisition data dictionary csv for column names
    acq_data_dict_fp = './data/acquisition_data_dict_summary.csv'
    list_acq_col_names = pd.read_csv(acq_data_dict_fp)['Field Name'].tolist()

    # load acquisition data dictionary csv for column names
    perf_data_dict_fp = './data/performance_data_dict_summary.csv'
    list_perf_col_names = pd.read_csv(perf_data_dict_fp)['Field Name'].tolist()

    # load acquisition csv
    ## string for fp needs variable
    acq_data_fp = f'./data/2005Q1/Acquisition_{year}Q{quarter}.txt'
    df_acq = pd.read_csv(acq_data_fp, sep='|', header=None, names=list_acq_col_names)
    
    # load performance csv
    perf_data_fp = f'./data/2005Q1/Performance_{year}Q{quarter}.txt'
    df_perf = pd.read_csv(perf_data_fp, sep='|', header=None, names=list_perf_col_names)
    
    return df_acq, df_perf

In [4]:
df_acq, df_perf = load_dataset()

In [5]:
df_acq.head()

,LOAN IDENTIFIER,ORIGINATION CHANNEL,SELLER NAME,ORIGINAL INTEREST RATE,ORIGINAL UPB,ORIGINAL LOAN TERM,ORIGINATION DATE,FIRST PAYMENT DATE,ORIGINAL LOAN-TO-VALUE (LTV),ORIGINAL COMBINED LOAN-TO-VALUE (CLTV),...,PROPERTY TYPE,NUMBER OF UNITS,OCCUPANCY TYPE,PROPERTY STATE,ZIP CODE SHORT,PRIMARY MORTGAGE INSURANCE PERCENT,PRODUCT TYPE,CO-BORROWER CREDIT SCORE AT ORIGINATION,MORTGAGE INSURANCE TYPE,RELOCATION MORTGAGE INDICATOR
0,100000102115,C,"CITIMORTGAGE, INC.",4.500,95000,120,02/2005,04/2005,51,51.0,...,SF,1,P,TX,750,NaN,FRM,783.0,NaN,N
1,100001329224,B,"BANK OF AMERICA, N.A.",5.875,141000,360,12/2004,02/2005,78,78.0,...,SF,1,P,IL,606,NaN,FRM,733.0,NaN,N
2,100004036273,C,"BANK OF AMERICA, N.A.",6.625,139000,360,01/2005,03/2005,95,95.0,...,SF,1,S,FL,349,30.0,FRM,NaN,1.0,N
3,100004945019,R,"BANK OF AMERICA, N.A.",5.875,104000,360,12/2004,02/2005,68,68.0,...,PU,1,P,SC,295,NaN,FRM,NaN,NaN,N
4,100013634177,C,"CITIMORTGAGE, INC.",6.250,51000,240,12/2004,02/2005,69,69.0,...,SF,1,P,WI,544,NaN,FRM,NaN,NaN,N


In [10]:
df_perf.head()

,LOAN IDENTIFIER,MONTHLY REPORTING PERIOD,SERVICER NAME,CURRENT INTEREST RATE,CURRENT ACTUAL UPB,LOAN AGE,REMAINING MONTHS TO LEGAL MATURITY,ADJUSTED MONTHS TO MATURITY,MATURITY DATE,METROPOLITAN STATISTICAL AREA (MSA),...,ASSOCIATED TAXES FOR HOLDING PROPERTY,NET SALE PROCEEDS,CREDIT ENHANCEMENT PROCEEDS,REPURCHASE MAKE WHOLE PROCEEDS,OTHER FORECLOSURE PROCEEDS,NON INTEREST BEARING UPB,PRINCIPAL FORGIVENESS AMOUNT,REPURCHASE MAKE WHOLE PROCEEDS FLAG,FORECLOSURE PRINCIPAL WRITE-OFF AMOUNT,SERVICING ACTIVITY INDICATOR
0,100000102115,03/01/2005,"CITIMORTGAGE ASSET MANAGEMENT, INC.",4.5,NaN,0,120,119.0,03/2015,43300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100000102115,04/01/2005,NaN,4.5,NaN,1,119,118.0,03/2015,43300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100000102115,05/01/2005,NaN,4.5,NaN,2,118,117.0,03/2015,43300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100000102115,06/01/2005,NaN,4.5,NaN,3,117,116.0,03/2015,43300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100000102115,07/01/2005,NaN,4.5,NaN,4,116,115.0,03/2015,43300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_acq.describe()

,LOAN IDENTIFIER,ORIGINAL INTEREST RATE,ORIGINAL UPB,ORIGINAL LOAN TERM,ORIGINAL LOAN-TO-VALUE (LTV),ORIGINAL COMBINED LOAN-TO-VALUE (CLTV),NUMBER OF BORROWERS,ORIGINAL DEBT TO INCOME RATIO,BORROWER CREDIT SCORE AT ORIGINATION,NUMBER OF UNITS,ZIP CODE SHORT,PRIMARY MORTGAGE INSURANCE PERCENT,CO-BORROWER CREDIT SCORE AT ORIGINATION,MORTGAGE INSURANCE TYPE
count,3.036210e+05,303621.000000,303621.000000,303621.000000,303621.000000,303605.000000,303614.000000,282264.000000,301180.000000,303621.000000,303621.000000,40476.000000,145381.000000,40476.000000
mean,5.497815e+11,5.674627,162760.751068,311.845834,68.337019,69.981634,1.585787,36.042467,723.140836,1.039151,510.137994,22.689791,730.864315,1.041037
std,2.597903e+11,0.369377,83214.594649,79.046352,17.460744,18.134376,0.506746,13.160504,58.338406,0.247905,304.250841,7.225863,57.286263,0.198377
min,1.000001e+11,2.990000,7000.000000,60.000000,1.000000,1.000000,1.000000,1.000000,333.000000,1.000000,0.000000,6.000000,300.000000,1.000000
25%,3.250128e+11,5.500000,99000.000000,240.000000,58.000000,59.000000,1.000000,26.000000,680.000000,1.000000,275.000000,17.000000,691.000000,1.000000
50%,5.497583e+11,5.750000,147000.000000,360.000000,73.000000,74.000000,2.000000,36.000000,730.000000,1.000000,484.000000,25.000000,741.000000,1.000000
75%,7.748965e+11,5.875000,214000.000000,360.000000,80.000000,80.000000,2.000000,46.000000,773.000000,1.000000,800.000000,25.000000,779.000000,1.000000
max,9.999962e+11,9.000000,692000.000000,360.000000,97.000000,193.000000,7.000000,64.000000,850.000000,4.000000,999.000000,40.000000,842.000000,2.000000


In [12]:
# add write up on above


In [15]:
df_acq.dtypes

LOAN IDENTIFIER                              int64
ORIGINATION CHANNEL                         object
SELLER NAME                                 object
ORIGINAL INTEREST RATE                     float64
ORIGINAL UPB                                 int64
ORIGINAL LOAN TERM                           int64
ORIGINATION DATE                            object
FIRST PAYMENT DATE                          object
ORIGINAL LOAN-TO-VALUE (LTV)                 int64
ORIGINAL COMBINED LOAN-TO-VALUE (CLTV)     float64
NUMBER OF BORROWERS                        float64
ORIGINAL DEBT TO INCOME RATIO              float64
BORROWER CREDIT SCORE AT ORIGINATION       float64
FIRST TIME HOME BUYER INDICATOR             object
LOAN PURPOSE                                object
PROPERTY TYPE                               object
NUMBER OF UNITS                              int64
OCCUPANCY TYPE                              object
PROPERTY STATE                              object
ZIP CODE SHORT                 

In [9]:
# define function to clean and merge datasets
### needs write up on assumptions and decisions

# def clean_merge_datasets()

# condense df_perf down to last status of each loan
loan_ids = df_perf['LOAN IDENTIFIER'].tolist()

last_index = []
for i in range(1,len(loan_ids)):
    if loan_ids[i] != loan_ids[i-1]:
        last_index.append(i-1)
        
    if i == len(loan_ids)-1:
        last_index.append(i)

df_perf = df_perf.iloc[last_index]

# condense perf df down to loan id and zero balance code
df_perf = df_perf[['LOAN IDENTIFIER', 'ZERO BALANCE CODE']]

# rename target column to default
df_perf.rename({'ZERO BALANCE CODE': 'DEFAULT'}, axis=1, inplace=True)

# map zero balance codes to binary 
df_perf['DEFAULT'] = df_perf['DEFAULT'].map(lambda x: 1 if x in [2, 3, 6, 9, 15] else 0)

# 

TypeError: cannot perform reduce with flexible type